In [1]:
from pathlib import Path
import pickle
import yaml
from collections import Counter, defaultdict

In [2]:
num_classes = 80
label_path = Path("D:/ml/code/yolov5/data/coco128.yaml")

In [3]:
val_dir = Path("D:/ml/code/datasets/coco128/labels/train2017")
pred_dir = Path("D:/ml/code/yolov5/runs/detect/coco128/yolov5s/labels")

In [4]:
assert label_path.exists(), f"{label_path} not exists"
assert val_dir.exists(), f"{val_dir} not exists"
assert pred_dir.exists(), f"{pred_dir} not exists"

# 创建id2txt的字典

```python
{
    0: ["pic1", "pic2", "pic3"...],
    1: ["pic2", "pic3", "pic5"...],
    ...
}
```

In [5]:
val_id2num = defaultdict(list)
pred_id2num = defaultdict(list)
val_id2num, pred_id2num

{0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}, 9: {}, 10: {}, 11: {}, 12: {}, 13: {}, 14: {}, 15: {}, 16: {}, 17: {}, 18: {}, 19: {}, 20: {}, 21: {}, 22: {}, 23: {}, 24: {}, 25: {}, 26: {}, 27: {}, 28: {}, 29: {}, 30: {}, 31: {}, 32: {}, 33: {}, 34: {}, 35: {}, 36: {}, 37: {}, 38: {}, 39: {}, 40: {}, 41: {}, 42: {}, 43: {}, 44: {}, 45: {}, 46: {}, 47: {}, 48: {}, 49: {}, 50: {}, 51: {}, 52: {}, 53: {}, 54: {}, 55: {}, 56: {}, 57: {}, 58: {}, 59: {}, 60: {}, 61: {}, 62: {}, 63: {}, 64: {}, 65: {}, 66: {}, 67: {}, 68: {}, 69: {}, 70: {}, 71: {}, 72: {}, 73: {}, 74: {}, 75: {}, 76: {}, 77: {}, 78: {}, 79: {}}
{0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}, 9: {}, 10: {}, 11: {}, 12: {}, 13: {}, 14: {}, 15: {}, 16: {}, 17: {}, 18: {}, 19: {}, 20: {}, 21: {}, 22: {}, 23: {}, 24: {}, 25: {}, 26: {}, 27: {}, 28: {}, 29: {}, 30: {}, 31: {}, 32: {}, 33: {}, 34: {}, 35: {}, 36: {}, 37: {}, 38: {}, 39: {}, 40: {}, 41: {}, 42: {}, 43: {}, 44: {}, 45: {}, 46: {}, 47

# 读取txt中的数据

In [6]:
def get_txts_data(path: Path) -> dict[str, list[str]]:
    """获取txt中数据,按照行分割

    Args:
        path (Path): txt文件夹路径

    Returns:
        dict[str, list[str]]:
            {
                img1: [line1],
                img2: [line1, line2],
                ....
            }
    """
    val_txts = path.glob("*.txt")
    data = {}
    for txt in val_txts:
        stem = txt.stem
        with open(txt) as f:
            lines = f.read().splitlines()                   # splitlines可以去除单个空行
            lines = [line for line in lines if line != ""]  # 去除空行
        data[stem] = lines
    return data

In [7]:
val_data = get_txts_data(val_dir)
print(len(val_data.keys()))
k = list(val_data.keys())[0]
print(k, ":", val_data[k])

128
000000000009 : ['45 0.479492 0.688771 0.955609 0.5955', '45 0.736516 0.247188 0.498875 0.476417', '50 0.637063 0.732938 0.494125 0.510583', '45 0.339438 0.418896 0.678875 0.7815', '49 0.646836 0.132552 0.118047 0.096937', '49 0.773148 0.129802 0.090734 0.097229', '49 0.668297 0.226906 0.131281 0.146896', '49 0.642859 0.079219 0.148063 0.148062']


In [8]:
pred_data = get_txts_data(pred_dir)
print(len(pred_data.keys()))
k = list(pred_data.keys())[0]
print(k, ":", pred_data[k])

126
000000000009 : ['60 0.486719 0.50625 0.973437 0.9875 0.27229', '50 0.589063 0.928125 0.15625 0.14375 0.336526', '45 0.642187 0.728125 0.503125 0.53125 0.338169', '50 0.647656 0.736458 0.495313 0.527083 0.507882', '45 0.75625 0.248958 0.453125 0.497917 0.776129']


# 获取 ids num

In [9]:
def get_txt_ids_num(data: dict[str, list[str]]) -> dict[str, dict[int, int]]:
    """获取每张图片中的类别id及其数量

    Args:
        data (dict[str, list[str]]): get_txts_data 的返回值

    Returns:
        dict[str, dict[int, int]]:
            {
                img1: {1: 3, 2: 1},
                img2: {1: 2, 3: 2, 4: 1},
                ...
            }
    """
    data_num = {}
    # key: filename
    # value: [line1, line2...]
    for stem, lines in data.items():
        # get line [0] label
        ids = sorted([int(line.split(" ")[0]) for line in lines])
        # get label num
        ids_num = dict(Counter(ids))
        data_num[stem] = ids_num
    return data_num

In [10]:
val_txt_ids_num = get_txt_ids_num(val_data)
print(len(val_txt_ids_num.keys()))
k = list(val_txt_ids_num.keys())[0]
print(k, ":", val_txt_ids_num[k])

128
000000000009 : {45: 3, 49: 4, 50: 1}


In [11]:
pred_txt_ids_num = get_txt_ids_num(pred_data)
print(len(pred_txt_ids_num.keys()))
k = list(pred_txt_ids_num.keys())[0]
print(k, ":", pred_txt_ids_num[k])

126
000000000009 : {45: 2, 50: 2, 60: 1}


# 将txt中的数据转移到id2txt中

In [12]:
val_id2num[0], val_id2num[1]

({}, {})

In [13]:
def txt_id2num(txt_ids_num: dict[str, dict[int, int]], id2num: dict[int, dict[str, int]]):
    """将uniqueid中的图片数据放入

    Args:
        txt_unique_ids (dict[str, dict[int, int]]): get_txt_ids_num 的返回值
        id2num (dict[int, dict[int, dict[str, int]]):
            {
                0: {img1: 2, img2: 3...},
                1: {img1: 1, img3: 2...},
                ...
            }
    """
    # 清空id2txt中数组的内容
    for i in id2num.values():
        i.clear()
    # stem: filename
    # id_num: {id0: num1, id1: num2...}
    for stem, id_num in txt_ids_num.items():
        for id, num in id_num.items():
            # {id0: {img1: num1, img2: num2}}
            id2num[id][stem] = num

In [14]:
txt_id2num(val_txt_ids_num, val_id2num)
val_id2num[1]

{'000000000074': 1, '000000000531': 3, '000000000641': 2}

In [15]:
txt_id2num(pred_txt_ids_num, pred_id2num)
pred_id2num[1]

{'000000000074': 1, '000000000086': 1, '000000000641': 1}

In [16]:
for i in range(num_classes):
    print(len(val_id2num[i]), len(pred_id2num[i]))

61 62
3 3
12 10
4 4
5 5
5 5
3 2
5 4
2 1
4 2
0 0
2 2
0 0
5 4
2 2
4 5
9 7
1 1
0 0
0 0
4 4
1 1
2 2
4 4
4 4
4 4
9 3
6 4
2 2
5 5
1 2
2 1
6 4
2 2
4 3
4 4
3 2
0 0
5 5
6 8
5 6
10 9
6 3
7 5
5 4
9 9
1 1
0 0
2 0
1 0
4 5
3 3
1 1
5 5
2 2
4 5
9 7
5 5
9 9
3 3
10 9
2 1
2 3
2 0
2 0
5 3
0 0
5 4
3 3
5 5
0 0
4 3
5 5
6 5
8 7
2 4
1 0
6 5
0 0
2 1


# 每张图片一次对比val和pred的每个类别的数量

In [17]:
def get_metrics(val_id2num: dict[int, dict[str, int]], pred_id2num: dict[int, dict[str, int]]) -> dict[int, dict]:
    """获取每个类别和总的评估指标

    Args:
        val_id2num (dict[int, list[dict[str, int]]]):  val
        predl_id2num (dict[int, list[dict[str, int]]]): pred

    Returns:
        dict[int, dict]
    """
    metrics = {}
    for i in range(num_classes):
        val_stem_num_for_single_id = val_id2num[i]
        pred_stem_num_for_single_id = pred_id2num[i]

        # set在这里不为去重,仅仅为了求交集,差集
        val_stem_for_single_id: set = set(val_stem_num_for_single_id.keys())   # 单一类别val图片名字
        pred_stem_for_single_id: set = set(pred_stem_num_for_single_id.keys()) # 单一类别pred图片名字

        # 检测到的图片名字
        detect      = val_stem_for_single_id.intersection(pred_stem_for_single_id)
        # 没有检测到图片名字
        not_detect  = val_stem_for_single_id.difference(pred_stem_for_single_id)
        # 过度检测到图片名字
        over_detect = pred_stem_for_single_id.difference(val_stem_for_single_id)

        # 每个图片检测到的数量的差异
        # {img1: 1, img2: 0, img3: -1}
        # 0代表检测数量正常,正数代表检测多了,负数代表检测少了
        deviation_for_stem = {}
        # 正常检测到
        for det in detect:
            # pred - val: 0代表检测数量正常,正数代表检测多了,负数代表检测少了
            deviation_for_stem[det] = pred_stem_num_for_single_id[det] - val_stem_num_for_single_id[det]
        # 没有检测到
        for det in not_detect:
            deviation_for_stem[det] = -val_stem_num_for_single_id[det]
        # 过度检测到(是否该计入总分有疑点)
        for det in over_detect:
            deviation_for_stem[det] = pred_stem_num_for_single_id[det]
        # print(deviation_for_stem)

        # 获取各个差异的总数
        # {0: 10, 1: 2, -1: 3}
        deviation_count = dict(Counter(deviation_for_stem.values()))
        # 按照key排序
        # {-1: 3, 0: 10, 1: 2}
        deviation_count = dict(sorted(deviation_count.items(), key=lambda x: x[0]))
        # print(deviation_count)
        # 总检测数量
        deviation_sum = sum(deviation_count.values())
        if deviation_sum == 0:  # 防止除0
            deviation_sum = 0.1

        # 各个差异占总的百分比
        # {-1: 0.15, 0: 0.7, 1: 0.15}
        deviation_ratio = {}
        for deviation, count in deviation_count.items():
            deviation_ratio[deviation] = count / deviation_sum
        # print(deviation_ratio)
        metrics[i] = {
            "deviation_for_stem": deviation_for_stem,
            "deviation_count": deviation_count,
            "deviation_ratio": deviation_ratio,
        }
    return metrics

In [18]:
metrics = get_metrics(val_id2num, pred_id2num)
metrics

{0: {'deviation_for_stem': {'000000000415': 1,
   '000000000641': -6,
   '000000000370': 0,
   '000000000357': -6,
   '000000000572': 0,
   '000000000326': 0,
   '000000000395': 0,
   '000000000529': -1,
   '000000000308': 0,
   '000000000397': 0,
   '000000000322': 0,
   '000000000136': 0,
   '000000000419': 0,
   '000000000446': 0,
   '000000000077': -1,
   '000000000542': -2,
   '000000000294': 0,
   '000000000520': -2,
   '000000000623': 0,
   '000000000113': 0,
   '000000000192': 0,
   '000000000564': 0,
   '000000000074': 4,
   '000000000165': 0,
   '000000000536': 0,
   '000000000049': 2,
   '000000000086': 0,
   '000000000360': 0,
   '000000000389': -2,
   '000000000531': 2,
   '000000000328': 0,
   '000000000544': -4,
   '000000000315': -5,
   '000000000368': -1,
   '000000000382': 1,
   '000000000569': 0,
   '000000000338': 0,
   '000000000510': 0,
   '000000000201': 0,
   '000000000257': 4,
   '000000000241': 1,
   '000000000151': 0,
   '000000000474': 0,
   '000000000532': 

In [19]:
metrics[0]

{'deviation_for_stem': {'000000000415': 1,
  '000000000641': -6,
  '000000000370': 0,
  '000000000357': -6,
  '000000000572': 0,
  '000000000326': 0,
  '000000000395': 0,
  '000000000529': -1,
  '000000000308': 0,
  '000000000397': 0,
  '000000000322': 0,
  '000000000136': 0,
  '000000000419': 0,
  '000000000446': 0,
  '000000000077': -1,
  '000000000542': -2,
  '000000000294': 0,
  '000000000520': -2,
  '000000000623': 0,
  '000000000113': 0,
  '000000000192': 0,
  '000000000564': 0,
  '000000000074': 4,
  '000000000165': 0,
  '000000000536': 0,
  '000000000049': 2,
  '000000000086': 0,
  '000000000360': 0,
  '000000000389': -2,
  '000000000531': 2,
  '000000000328': 0,
  '000000000544': -4,
  '000000000315': -5,
  '000000000368': -1,
  '000000000382': 1,
  '000000000569': 0,
  '000000000338': 0,
  '000000000510': 0,
  '000000000201': 0,
  '000000000257': 4,
  '000000000241': 1,
  '000000000151': 0,
  '000000000474': 0,
  '000000000532': -2,
  '000000000110': 0,
  '000000000589': 0,
 

# load label name

In [20]:
with open(label_path, 'r', encoding='utf-8') as f:
    dataset = yaml.safe_load(f)
names = dataset["names"]
names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

# pandas

In [21]:
import pandas as pd

In [22]:
dfs: list[pd.DataFrame] = []
for i in range(num_classes):
    metric = metrics[i]
    metric.pop("deviation_for_stem")    # 去除每张图片的偏差值
    df = pd.DataFrame(metrics[i])
    df["deviation"] = df.index
    df["class"] = names[i]
    # 重新排序
    df = df[["class", "deviation", "deviation_count", "deviation_ratio"]]
    df.index = range(df.shape[0])
    dfs.append(df)

In [23]:
dfs[0]
# - 代表检测少
# 0 代表检测正确
# + 代表检测多

,class,deviation,deviation_count,deviation_ratio
0,person,-6,3,0.046875
1,person,-5,1,0.015625
2,person,-4,1,0.015625
3,person,-2,4,0.062500
4,person,-1,7,0.109375
5,person,0,35,0.546875
6,person,1,8,0.125000
7,person,2,3,0.046875
8,person,4,2,0.031250


In [24]:
dfs[1]
# - 代表检测少
# 0 代表检测正确
# + 代表检测多

,class,deviation,deviation_count,deviation_ratio
0,bicycle,-3,1,0.25
1,bicycle,-1,1,0.25
2,bicycle,0,1,0.25
3,bicycle,1,1,0.25
